In [1]:
# For Google Colab use
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd '/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder'    
except ModuleNotFoundError:
    pass

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder


In [2]:
# Imports
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from numpy.random import seed

import tensorflow as tf

import keras
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import *
from keras.applications import *
from keras import metrics
from keras.losses import binary_crossentropy
from keras import backend as K

# !pip install -U keras-tuner
# from kerastuner.tuners import RandomSearch, Hyperband
# from kerastuner.engine.hypermodel import HyperModel
# from kerastuner.engine.hyperparameters import HyperParameters
# from kerastuner import Objective

In [3]:
# Check GPU available
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Found GPU at: /device:GPU:0
Num GPUs Available:  1


In [4]:
# General model settings
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 100, 100, 3
EPOCHS = 1
DATA_GENERATOR_SEED = 1337
BATCH_SIZE = 192
VALIDATION_SPLIT = 0.1
LATENT_DIM = 20000
tf.random.set_seed(DATA_GENERATOR_SEED)
seed(DATA_GENERATOR_SEED)

# Pick dataset; DF_TYPE={'rnd-30', 'avg-30'}
DF_TYPE = 'rnd-30'
TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}'
TEST_DIR = f'{TRAIN_VAL_DIR}-test'

TRAIN_VAL_DIR += '/Celeb-real/'

TRAIN_VAL_DIR = './Celeb-DF-v2/Celeb-TEMP'
# TEST_DIR += '/Celeb-real'
TRAIN_VAL_DIR, TEST_DIR

('./Celeb-DF-v2/Celeb-TEMP', './Celeb-DF-v2/Celeb-rnd-30-test')

In [5]:
# !ln -s "/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder/Celeb-DF-v2/Celeb-avg-30/Celeb-real" "/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder/Celeb-DF-v2/Celeb-TEMP/sym"
# !ls "/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder/Celeb-DF-v2/Celeb-TEMP"

In [6]:
TRAIN_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, horizontal_flip = True, fill_mode='nearest', validation_split = VALIDATION_SPLIT)
TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                    batch_size = BATCH_SIZE,
                                                    class_mode = 'input', 
                                                    target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                    subset = 'training',
                                                    seed = DATA_GENERATOR_SEED,
                                                    follow_links = True)

VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, validation_split = VALIDATION_SPLIT)
VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                         batch_size = BATCH_SIZE,
                                                         class_mode = 'input', 
                                                         target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                         subset = 'validation',
                                                         seed = DATA_GENERATOR_SEED)

# TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0)
# TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
#                                                   batch_size = BATCH_SIZE,
#                                                   class_mode = 'binary', 
#                                                   target_size = (IMG_HEIGHT, IMG_WIDTH),                                
#                                                   seed = DATA_GENERATOR_SEED)

Found 5065 images belonging to 1 classes.
Found 562 images belonging to 1 classes.


In [7]:
# OC-FakeDect Encoder

encoder_inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
# 100x100x3
x = layers.Conv2D(16, 3, strides=1, padding="same")(encoder_inputs)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 100x100x16
x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 50x50x32
x = layers.Conv2D(64, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 50x50x64
x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 25x25x32
x = layers.Flatten()(x)
# 20000

# Sampling Class, Random sampling function for latent vector, z; "variational part"
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
# Sampling
z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

# Defining full encoder as keras model
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 100, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100, 100, 16) 400         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 100, 100, 16) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [8]:
# OC-FakeDect Decoder

latent_inputs = keras.Input(shape=(LATENT_DIM,))
# 20000
x = layers.Reshape((25, 25, 32))(latent_inputs)
x = layers.Conv2DTranspose(32, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 25x25x32
x = layers.Conv2DTranspose(64, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 50x50x64
x = layers.Conv2DTranspose(32, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 50x50x32
x = layers.Conv2DTranspose(16, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 100x100x16
x = layers.Conv2DTranspose(3, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
decoder_outputs = layers.Activation('sigmoid')(x)
# 100x100x3

# Defining full decoder as keras model
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
reshape (Reshape)            (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 25, 25, 32)        9248      
_________________________________________________________________
batch_normalization_4 (Batch (None, 25, 25, 32)        100       
_________________________________________________________________
activation_4 (Activation)    (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 50, 50, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 50, 50, 64)        200 

In [9]:
# Build VAE model as custom (keras model class) class
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker]

    def call(self, inputs):
        x = inputs[0]
        X = np.expand_dims(x, axis=0)
        z_mean, z_log_var, z = encoder(X)
        reconstruction = decoder(z)
        z_mean, z_log_var, z = encoder(X)
        reconstruction = decoder(z)
        reconstruction_loss = tf.reduce_mean(keras.losses.binary_crossentropy(inputs, reconstruction))
        reconstruction_loss *= 28 * 28
        kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        kl_loss = tf.reduce_mean(kl_loss)
        kl_loss *= -0.5
        total_loss = reconstruction_loss + kl_loss
        self.add_metric(kl_loss, name='kl_loss', aggregation='mean')
        self.add_metric(total_loss, name='total_loss', aggregation='mean')
        self.add_metric(reconstruction_loss, name='reconstruction_loss', aggregation='mean')
        return reconstruction

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(binary_crossentropy(data, reconstruction),
                              axis=(1, 2)))
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {"loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result()}

    def test_step(self, data):
        if isinstance(data, tuple):
            data = data[0]

        z_mean, z_log_var, z = encoder(data)
        reconstruction = decoder(z)
        reconstruction_loss = tf.reduce_mean(
            keras.losses.binary_crossentropy(data, reconstruction)
        )
        reconstruction_loss *= 28 * 28
        kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        kl_loss = tf.reduce_mean(kl_loss)
        kl_loss *= -0.5
        total_loss = reconstruction_loss + kl_loss
        return {"loss": total_loss,
                "reconstruction_loss": reconstruction_loss,
                "kl_loss": kl_loss}

In [10]:
# Define callbacks e.g. Early Stopping
EARLY_STOP = EarlyStopping(monitor='reconstruction_loss',
                           patience=1,
                           mode='min',
                           verbose=1,
                           restore_best_weights=True)

# Define Model
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [ ]:
# Trains for epochs, also very slow
for _ in tqdm(range(EPOCHS)):
    for _ in tqdm(range(TRAIN_GENERATOR.n//BATCH_SIZE+1)):
        vae.fit(np.concatenate([TRAIN_GENERATOR.next()[0], VALIDATION_GENERATOR.next()[0]], axis=0),
                epochs=100,
                batch_size=2*BATCH_SIZE,
                verbose=1,
                callbacks=[EARLY_STOP])
    # vae.save_weights('./Checkpoints/OC-FakeDect/')

  0%|          | 0/27 [00:00<?, ?it/s]

Epoch 1/100
1/1 [==============================] - 11s 11s/step - loss: 6928.9902 - reconstruction_loss: 6919.6655 - kl_loss: 9.3248
Epoch 2/100
1/1 [==============================] - 1s 561ms/step - loss: 6914.2617 - reconstruction_loss: 6865.1274 - kl_loss: 49.1341
Epoch 3/100
1/1 [==============================] - 1s 562ms/step - loss: 6810.4653 - reconstruction_loss: 6809.5688 - kl_loss: 0.8967
Epoch 4/100
1/1 [==============================] - 1s 562ms/step - loss: 6748.2773 - reconstruction_loss: 6748.2583 - kl_loss: 0.0192
Epoch 5/100
1/1 [==============================] - 1s 559ms/step - loss: 6684.2803 - reconstruction_loss: 6684.2578 - kl_loss: 0.0223
Epoch 6/100
1/1 [==============================] - 1s 555ms/step - loss: 6629.2915 - reconstruction_loss: 6629.2539 - kl_loss: 0.0374
Epoch 7/100
1/1 [==============================] - 1s 560ms/step - loss: 6599.9131 - reconstruction_loss: 6599.7476 - kl_loss: 0.1656
Epoch 8/100
1/1 [==============================] - 1s 561ms/st


  4%|▎         | 1/27 [01:37<42:18, 97.64s/it]

Epoch 00026: early stopping
Epoch 1/100
1/1 [==============================] - 1s 615ms/step - loss: 6379.9492 - reconstruction_loss: 6350.4458 - kl_loss: 29.5037
Epoch 2/100
1/1 [==============================] - 1s 566ms/step - loss: 6383.1426 - reconstruction_loss: 6325.8047 - kl_loss: 57.3377
Epoch 3/100
1/1 [==============================] - 1s 561ms/step - loss: 6363.7876 - reconstruction_loss: 6317.8530 - kl_loss: 45.9347
Epoch 4/100
1/1 [==============================] - 1s 561ms/step - loss: 6360.4683 - reconstruction_loss: 6327.6226 - kl_loss: 32.8456
Restoring model weights from the end of the best epoch.



  7%|▋         | 2/27 [02:25<34:25, 82.64s/it]

Epoch 00004: early stopping
Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 6430.1172 - reconstruction_loss: 6394.2324 - kl_loss: 35.8846
Epoch 2/100
1/1 [==============================] - 1s 548ms/step - loss: 6412.1865 - reconstruction_loss: 6355.8770 - kl_loss: 56.3097
Epoch 3/100
1/1 [==============================] - 1s 556ms/step - loss: 6414.1631 - reconstruction_loss: 6335.9053 - kl_loss: 78.2579
Epoch 4/100
1/1 [==============================] - 1s 539ms/step - loss: 6394.1807 - reconstruction_loss: 6335.9150 - kl_loss: 58.2658
Restoring model weights from the end of the best epoch.



 11%|█         | 3/27 [02:38<24:46, 61.95s/it]

Epoch 00004: early stopping


In [ ]:
# Plot 5 images from test set
(x_train, _), (x_test, _) = TRAIN_GENERATOR.next(), VALIDATION_GENERATOR.next()

for x in x_train[-3:]:
    X = np.expand_dims(x, axis=0)
    Z_mean, Z_log_var, Z = vae.encoder.predict(X)
    X_prime = vae.decoder.predict(Z)
    face = np.array(X_prime.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)*255, dtype=np.uint8)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 5))
    axes[0].imshow(X.squeeze())
    axes[1].imshow(face)
    fig.tight_layout()

for x in x_test[-3:]:
    X = np.expand_dims(x, axis=0)
    Z_mean, Z_log_var, Z = vae.encoder.predict(X)
    X_prime = vae.decoder.predict(Z)
    face = np.array(X_prime.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)*255, dtype=np.uint8)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 5))
    axes[0].imshow(X.squeeze())
    axes[1].imshow(face)
    fig.tight_layout()

In [ ]:
# # Load all training data, takes a very long time (3hrs)
# training_data = []
# for i in tqdm(range(TRAIN_GENERATOR.n//BATCH_SIZE+1)):
#     (x_train, _), (x_test, _) = TRAIN_GENERATOR.next(), VALIDATION_GENERATOR.next()
#     training_data.extend(np.concatenate([x_train, x_test], axis=0)) 

# vae.fit(training_data,
#         epochs=100,
#         batch_size=2*BATCH_SIZE,
#         verbose=1)

In [ ]:
# # Trains model for 1 step (i.e. one batch, not an epoch)
# (x_train, _), (x_test, _) = TRAIN_GENERATOR.next(), VALIDATION_GENERATOR.next()
# training_data = np.concatenate([x_train, x_test], axis=0)
# vae.fit(training_data,
#         epochs=100,
#         batch_size=2*BATCH_SIZE,
#         verbose=1)

In [ ]:
# OLD Autoencoder (for use w/ 28x28 images, used for testing)

# encoder_inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
# x = layers.Convolution2D(64, (3, 3), padding = 'same')(encoder_inputs)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Convolution2D(64, (3, 3), padding = 'same')(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.AveragePooling2D()(x)
# x = layers.Convolution2D(128, (3, 3), padding = 'same')(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Convolution2D(128, (3, 3), padding = 'same')(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.AveragePooling2D()(x)
# x = layers.Conv2D(64, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Conv2D(64, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Flatten()(x)
# x = layers.Dense(128)(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Dense(128)(x)
# x = layers.Activation('tanh')(x)
# z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
# z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)
# z = Sampling()([z_mean, z_log_var])
# encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
# latent_inputs = keras.Input(shape=(LATENT_DIM,))
# x = layers.Dense(49)(latent_inputs)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)
# x = layers.Reshape((7, 7, 1))(x)
# x = layers.Convolution2D(128, (3, 3), padding = 'same')(x)
# x = layers.BatchNormalization(axis=1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Convolution2D(128, (3, 3), padding = 'same')(x)
# x = layers.BatchNormalization(axis=1)(x)
# x = layers.Activation('relu')(x)
# x = layers.UpSampling2D()(x)
# x = layers.Convolution2D(64, (3, 3), padding = 'same')(x)
# x = layers.BatchNormalization(axis=1)(x)
# x = layers.Activation('relu')(x)
# x = layers.Convolution2D(64, (3, 3), padding = 'same')(x)
# x = layers.BatchNormalization(axis=1)(x)
# x = layers.Activation('relu')(x)
# x = layers.UpSampling2D()(x)
# decoder_outputs = layers.Convolution2D(3, (3, 3), padding = 'same', activation = 'tanh')(x)
# decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

In [ ]:
# (x_train, _) = TRAIN_GENERATOR.next()
# (x_test, _) = VALIDATION_GENERATOR.next()
# training_data = np.concatenate([x_train, x_test], axis=0)
# training_data = np.expand_dims(training_data, -1).astype("float32")
# training_data = training_data[:, :, :, :, 0]